# Play with keras model

In [27]:
%pylab inline
import numpy as np
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from preprocess import loadPreData

'''
    This demonstrates how to reach a 0.80 ROC AUC score (local 4-fold validation)
    in the Kaggle Nile virus prediction challenge. 

    The model trains in a few seconds on CPU.
'''

Populating the interactive namespace from numpy and matplotlib


//anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


'\n    This demonstrates how to reach a 0.80 ROC AUC score (local 4-fold validation)\n    in the Kaggle Nile virus prediction challenge. \n\n    The model trains in a few seconds on CPU.\n'

In [4]:
data = loadPreData()

Index([u'Address', u'Species', u'Block', u'Street', u'Trap',
       u'AddressNumberAndStreet', u'Latitude', u'Longitude',
       u'AddressAccuracy', u'NumMosquitos', u'WnvPresent'],
      dtype='object')
Index([u'Id', u'Address', u'Species', u'Block', u'Street', u'Trap',
       u'AddressNumberAndStreet', u'Latitude', u'Longitude',
       u'AddressAccuracy'],
      dtype='object')


In [69]:
X = data['train'].get_values().astype(float)
y = data['labels']
#y = np.array([y,1-y]).T
Y = np_utils.to_categorical(y)

(10506, 2)

In [67]:
input_dim = X.shape[1]
output_dim = 2

In [47]:
def scoreAUC(y,probs):
    ps = np.linspace(0.,1.,num=100)
    prs = []
    nrs = []
    for p in ps:
        preds = probs[:,0]<p
        pr = np.sum((y & preds))/float(np.sum(y))
        nr = np.sum((1-y & 1-preds))/float(np.sum(1-y))
        nrs.append(nr)
        prs.append(pr)
    xs = 1-np.array(nrs)
    ys = np.array(prs)
    dxs = xs[1:] - xs[:-1]
    ays = .5*(ys[1:] + ys[:-1])
    auc = np.sum(ays*dxs)
    return {'score':auc,'fpr':xs,'tpr':ys}

In [77]:
def build_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(output_dim))

    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    return model

In [78]:
def testModel():    
    X = np.random.rand(5000,5)
    y = (X[:,0]<0.5)*(X[:,1]<0.9)*(X[:,2]<0.85)
    X_valid = np.random.rand(1000,5)
    y_valid = (X_valid[:,0]<0.5)*(X_valid[:,1]<0.9)*(X_valid[:,2]<0.85)
    model = build_model(5, 1)
    model.fit(X, y, nb_epoch=10, batch_size=32, validation_data=(X_valid, y_valid), verbose=True)

In [79]:
nb_folds = 4
kfolds = KFold(len(y), nb_folds)
av_roc = 0.
f = 0
for train, valid in kfolds:
    print('---'*20)
    print('Fold', f)
    print('---'*20)
    f += 1
    X_train = X[train]
    X_valid = X[valid]
    Y_train = Y[train]
    Y_valid = Y[valid]
    y_valid = y[valid]

    print("Building model...")
    model = build_model(input_dim, output_dim)

    print("Training model...")

    model.fit(X_train, Y_train, nb_epoch=5, batch_size=32, validation_data=(X_valid, Y_valid), verbose=True)
    valid_preds = model.predict_proba(X_valid)
    valid_preds = valid_preds[:, 1]
    roc = metrics.roc_auc_score(y_valid, valid_preds)
    print("ROC:", roc)
    av_roc += roc

print('Average ROC:', av_roc/nb_folds)

------------------------------------------------------------
('Fold', 0)
------------------------------------------------------------
Building model...
Training model...
Train on 7879 samples, validate on 2627 samples
Epoch 1/5
7879/7879 [==============================] - 2s - loss: 4.5428 - acc: 0.7108 - val_loss: 1.1351 - val_acc: 0.9296
Epoch 2/5
7879/7879 [==============================] - 0s - loss: 1.8129 - acc: 0.8822 - val_loss: 1.1351 - val_acc: 0.9296
Epoch 3/5
7879/7879 [==============================] - 0s - loss: 0.9714 - acc: 0.9368 - val_loss: 1.1351 - val_acc: 0.9296
Epoch 4/5
7879/7879 [==============================] - 0s - loss: 0.8662 - acc: 0.9436 - val_loss: 1.1351 - val_acc: 0.9296
Epoch 5/5
1376/2627 [==============>...............] - ETA: 0s ('ROC:', 0.5)
------------------------------------------------------------
('Fold', 1)
------------------------------------------------------------
Building model...
Training model...
Train on 7879 samples, validate on 2627

In [59]:
model = build_model(input_dim, output_dim)
model.fit(X, y, nb_epoch=20, batch_size=16, verbose=1)

Epoch 1/20
10506/10506 [==============================] - 2s - loss: 2.4059 - acc: 0.8463     
Epoch 2/20
10506/10506 [==============================] - 2s - loss: 0.9775 - acc: 0.9388     
Epoch 3/20
10506/10506 [==============================] - 2s - loss: 0.8847 - acc: 0.9449     
Epoch 4/20
10506/10506 [==============================] - 2s - loss: 0.8652 - acc: 0.9461     
Epoch 5/20
10506/10506 [==============================] - 2s - loss: 0.8561 - acc: 0.9468     
Epoch 6/20
10506/10506 [==============================] - 2s - loss: 0.8573 - acc: 0.9468     
Epoch 7/20
10506/10506 [==============================] - 2s - loss: 0.8596 - acc: 0.9465     
Epoch 8/20
10506/10506 [==============================] - 2s - loss: 0.8519 - acc: 0.9471     
Epoch 9/20
10506/10506 [==============================] - 2s - loss: 0.8499 - acc: 0.9473     
Epoch 10/20
10506/10506 [==============================] - 2s - loss: 0.8469 - acc: 0.9475     
Epoch 11/20
10506/10506 [========================